In [1]:
from google.colab import files
files.upload()


Saving kaggle (3).json to kaggle (3).json


{'kaggle (3).json': b'{"username":"marcelasuarez01","key":"96f3101f7d512cd123cb29b065f7c528"}'}

In [2]:
import os
import shutil

# Crear carpeta .kaggle
os.makedirs(".kaggle", exist_ok=True)

# Renombrar y mover el archivo subido
shutil.move("kaggle (3).json", ".kaggle/kaggle.json")

# Asignar permisos seguros
os.chmod(".kaggle/kaggle.json", 0o600)

# Definir la variable de entorno
os.environ['KAGGLE_CONFIG_DIR'] = ".kaggle"


In [3]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia



  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.13GB/s]


In [4]:
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import pandas as pd
import numpy as np

datos = pd.read_csv("train.csv")
datos.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [6]:
def revisar_valores_faltantes(df):
    # Identificar columnas con valores faltantes
    columnas_con_faltantes = datos.columns[df.isnull().any()]

    if len(columnas_con_faltantes) == 0:
        print("No hay valores faltantes en el DataFrame.")
    else:
        print("Columnas con valores faltantes:")
        for columna in columnas_con_faltantes:
            tipo_variable = df[columna].dtype
            if pd.api.types.is_numeric_dtype(tipo_variable):
                print(f"{columna}: Variable numérica, {df[columna].isnull().sum()} valores faltantes")
            else:
                print(f"{columna}: Variable no numérica, {df[columna].isnull().sum()} valores faltantes")

In [7]:
revisar_valores_faltantes(datos)

Columnas con valores faltantes:
ESTU_VALORMATRICULAUNIVERSIDAD: Variable no numérica, 6287 valores faltantes
ESTU_HORASSEMANATRABAJA: Variable no numérica, 30857 valores faltantes
FAMI_ESTRATOVIVIENDA: Variable no numérica, 32137 valores faltantes
FAMI_TIENEINTERNET: Variable no numérica, 26629 valores faltantes
FAMI_EDUCACIONPADRE: Variable no numérica, 23178 valores faltantes
FAMI_TIENELAVADORA: Variable no numérica, 39773 valores faltantes
FAMI_TIENEAUTOMOVIL: Variable no numérica, 43623 valores faltantes
ESTU_PAGOMATRICULAPROPIO: Variable no numérica, 6498 valores faltantes
FAMI_TIENECOMPUTADOR: Variable no numérica, 38103 valores faltantes
FAMI_TIENEINTERNET.1: Variable no numérica, 26629 valores faltantes
FAMI_EDUCACIONMADRE: Variable no numérica, 23664 valores faltantes


In [8]:
def convertir_si_no(valor):
    if isinstance(valor, str):  # Verifica si el valor es una cadena
        if valor.lower() == 'si' or valor.lower() == 's':
            return 1
        elif valor.lower() == 'no' or valor.lower() == 'n':
            return 0
    return valor  # Devuelve el valor original si no es 'si' ni 'no' o si no es una cadena

In [9]:
data_frame = datos.apply(lambda x: x.map(convertir_si_no))
data_frame

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,1.0,Técnica o tecnológica incompleta,1.0,...,0,0.0,1.0,1.0,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,0.0,Técnica o tecnológica completa,1.0,...,0,0.0,1.0,0.0,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,1.0,Secundaria (Bachillerato) completa,1.0,...,0,0.0,0.0,1.0,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,1.0,No sabe,1.0,...,0,0.0,1.0,1.0,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,1.0,Primaria completa,1.0,...,0,0.0,1.0,1.0,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,1.0,Secundaria (Bachillerato) completa,1.0,...,0,1.0,1.0,1.0,Secundaria (Bachillerato) incompleta,medio-alto,0.237,0.271,0.271,0.311
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,1.0,Primaria incompleta,1.0,...,0,0.0,1.0,1.0,Secundaria (Bachillerato) incompleta,bajo,0.314,0.240,0.278,0.260
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,1.0,Secundaria (Bachillerato) completa,1.0,...,0,1.0,1.0,1.0,Secundaria (Bachillerato) incompleta,medio-bajo,0.286,0.240,0.314,0.287
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,0.0,Primaria completa,0.0,...,0,1.0,1.0,0.0,Primaria completa,bajo,0.132,0.426,0.261,0.328


In [10]:
# Función para imputar valores faltantes según el tipo de variable
def imputar_valores(df):
    for columna in df.columns:
        if pd.api.types.is_numeric_dtype(df[columna]):
            # Imputación para variables cuantitativas (numéricas)
            df[columna] = df[columna].fillna(df[columna].median())  # Por ejemplo, podrías usar la mediana
        else:
            # Imputación para variables cualitativas (no numéricas)
            moda = df[columna].mode()[0]  # Obtenemos la moda para imputar
            df[columna] = df[columna].fillna(moda)
    return df

In [11]:
data_frame = imputar_valores(data_frame)
data_frame

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,1.0,Técnica o tecnológica incompleta,1.0,...,0,0.0,1.0,1.0,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,0.0,Técnica o tecnológica completa,1.0,...,0,0.0,1.0,0.0,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,1.0,Secundaria (Bachillerato) completa,1.0,...,0,0.0,0.0,1.0,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,1.0,No sabe,1.0,...,0,0.0,1.0,1.0,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,1.0,Primaria completa,1.0,...,0,0.0,1.0,1.0,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,1.0,Secundaria (Bachillerato) completa,1.0,...,0,1.0,1.0,1.0,Secundaria (Bachillerato) incompleta,medio-alto,0.237,0.271,0.271,0.311
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,1.0,Primaria incompleta,1.0,...,0,0.0,1.0,1.0,Secundaria (Bachillerato) incompleta,bajo,0.314,0.240,0.278,0.260
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,1.0,Secundaria (Bachillerato) completa,1.0,...,0,1.0,1.0,1.0,Secundaria (Bachillerato) incompleta,medio-bajo,0.286,0.240,0.314,0.287
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,0.0,Primaria completa,0.0,...,0,1.0,1.0,0.0,Primaria completa,bajo,0.132,0.426,0.261,0.328


In [12]:
revisar_valores_faltantes(data_frame)

No hay valores faltantes en el DataFrame.


In [13]:
columnas_float = data_frame.select_dtypes(include='float64').columns
columnas_float

Index(['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'coef_1', 'coef_2', 'coef_3', 'coef_4'],
      dtype='object')

In [14]:
data_frame[columnas_float] = data_frame[columnas_float].astype(int)
data_frame

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,1,Técnica o tecnológica incompleta,1,...,0,0,1,1,Postgrado,medio-alto,0,0,0,0
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,0,Técnica o tecnológica completa,1,...,0,0,1,0,Técnica o tecnológica incompleta,bajo,0,0,0,0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,1,Secundaria (Bachillerato) completa,1,...,0,0,0,1,Secundaria (Bachillerato) completa,bajo,0,0,0,0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,1,No sabe,1,...,0,0,1,1,Secundaria (Bachillerato) completa,alto,0,0,0,0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,1,Primaria completa,1,...,0,0,1,1,Primaria completa,medio-bajo,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,1,Secundaria (Bachillerato) completa,1,...,0,1,1,1,Secundaria (Bachillerato) incompleta,medio-alto,0,0,0,0
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,1,Primaria incompleta,1,...,0,0,1,1,Secundaria (Bachillerato) incompleta,bajo,0,0,0,0
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,1,Secundaria (Bachillerato) completa,1,...,0,1,1,1,Secundaria (Bachillerato) incompleta,medio-bajo,0,0,0,0
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,0,Primaria completa,0,...,0,1,1,0,Primaria completa,bajo,0,0,0,0


In [15]:
# Verificar filas duplicadas en el DataFrame
duplicated_rows = data_frame[data_frame.duplicated()]

# Mostrar las filas duplicadas, si las hay
if not duplicated_rows.empty:
    print("Hay filas duplicadas encontradas:")
    print(duplicated_rows)
else:
    print("No hay filas duplicadas en el DataFrame.")

No hay filas duplicadas en el DataFrame.


ONE HOT

In [16]:
# 1. Identificar las columnas que son variables cualitativas
columnas_cualitativas = data_frame.select_dtypes(include=['object']).columns

# 2. Realizar la conversión one-hot para las columnas cualitativas
data_frame_one_hot = pd.get_dummies(data_frame, columns=columnas_cualitativas)
len(data_frame_one_hot.columns.values)

1040

In [17]:
data_frame.select_dtypes(exclude=['object']).columns

Index(['ID', 'PERIODO', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA',
       'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'coef_1', 'coef_2', 'coef_3', 'coef_4'],
      dtype='object')

In [18]:
data_frame_one_hot.columns

Index(['ID', 'PERIODO', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA',
       'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'coef_1',
       ...
       'FAMI_EDUCACIONMADRE_Primaria completa',
       'FAMI_EDUCACIONMADRE_Primaria incompleta',
       'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa',
       'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta',
       'FAMI_EDUCACIONMADRE_Técnica o tecnológica completa',
       'FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta',
       'RENDIMIENTO_GLOBAL_alto', 'RENDIMIENTO_GLOBAL_bajo',
       'RENDIMIENTO_GLOBAL_medio-alto', 'RENDIMIENTO_GLOBAL_medio-bajo'],
      dtype='object', length=1040)

In [19]:
#Acá estaba comprobando los valores unicos de cada una de las columnas cualitativas, para comprobar el largo del dataframe_one hot
for columna in columnas_cualitativas:
    print(len(data_frame[columna].value_counts().index))

948
31
8
5
7
12
12
4


In [20]:
data_frame_one_hot

,ID,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,coef_1,...,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_alto,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,904256,20212,1,1,1,0,0,1,1,0,...,False,False,False,False,False,False,False,False,True,False
1,645256,20212,0,1,0,0,0,1,0,0,...,False,False,False,False,False,True,False,True,False,False
2,308367,20203,1,1,0,0,0,0,1,0,...,False,False,True,False,False,False,False,True,False,False
3,470353,20195,1,1,0,0,0,1,1,0,...,False,False,True,False,False,False,True,False,False,False
4,989032,20212,1,1,1,0,0,1,1,0,...,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,1,1,0,0,1,1,1,0,...,False,False,False,True,False,False,False,False,True,False
692496,754213,20212,1,1,0,0,0,1,1,0,...,False,False,False,True,False,False,False,True,False,False
692497,504185,20183,1,1,0,0,1,1,1,0,...,False,False,False,True,False,False,False,False,False,True
692498,986620,20195,0,0,0,0,1,1,0,0,...,True,False,False,False,False,False,False,True,False,False


In [21]:
# Función para convertir False a 0 y True a 1
def convertir_a_binario(valor):
    if valor == False:
        return 0
    elif valor == True:
        return 1
    else:
        return valor

In [22]:
# Aplicar la función a cada valor del DataFrame
data_frame = data_frame_one_hot.apply(lambda x: x.map(convertir_a_binario))
# Ahora, todos los valores binarios en el DataFrame se han convertido a 0 y 1


In [23]:
data_frame

,ID,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,coef_1,...,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_alto,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,904256,20212,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1,645256,20212,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
2,308367,20203,1,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
3,470353,20195,1,1,0,0,0,1,1,0,...,0,0,1,0,0,0,1,0,0,0
4,989032,20212,1,1,1,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,1,1,0,0,1,1,1,0,...,0,0,0,1,0,0,0,0,1,0
692496,754213,20212,1,1,0,0,0,1,1,0,...,0,0,0,1,0,0,0,1,0,0
692497,504185,20183,1,1,0,0,1,1,1,0,...,0,0,0,1,0,0,0,0,0,1
692498,986620,20195,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,1,0,0


In [24]:
data_frame.columns

Index(['ID', 'PERIODO', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA',
       'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'coef_1',
       ...
       'FAMI_EDUCACIONMADRE_Primaria completa',
       'FAMI_EDUCACIONMADRE_Primaria incompleta',
       'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa',
       'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta',
       'FAMI_EDUCACIONMADRE_Técnica o tecnológica completa',
       'FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta',
       'RENDIMIENTO_GLOBAL_alto', 'RENDIMIENTO_GLOBAL_bajo',
       'RENDIMIENTO_GLOBAL_medio-alto', 'RENDIMIENTO_GLOBAL_medio-bajo'],
      dtype='object', length=1040)

Normalizar datos numéricos

In [25]:
pip install scikit-learn

In [26]:
from sklearn.preprocessing import MinMaxScaler

# Inicializar el escalador para la escalación min-max
scaler_minmax = MinMaxScaler()

# Seleccionar solo las columnas numéricas para la normalización, excluyendo ID y PERIODO
columnas_numericas = data_frame.select_dtypes(include=['int64', 'float64']).columns.drop(['ID', 'PERIODO'])

# Aplicar la escalación min-max a las columnas numéricas, excluyendo ID y PERIODO
data_frame[columnas_numericas] = scaler_minmax.fit_transform(data_frame[columnas_numericas])

# Redondear los valores escalados a enteros
data_frame[columnas_numericas] = data_frame[columnas_numericas].round().astype(int)

In [27]:
# Calcular el valor mínimo y máximo de las columnas numéricas después de la escalación
min_values = data_frame[columnas_numericas].min()
max_values = data_frame[columnas_numericas].max()

# Imprimir los valores mínimo y máximo de las columnas numéricas
print("Valor mínimo de las columnas numéricas después de la escalación:\n", min_values)
print("\nValor máximo de las columnas numéricas después de la escalación:\n", max_values)


Valor mínimo de las columnas numéricas después de la escalación:
 FAMI_TIENEINTERNET                                      0
FAMI_TIENELAVADORA                                      0
FAMI_TIENEAUTOMOVIL                                     0
ESTU_PRIVADO_LIBERTAD                                   0
ESTU_PAGOMATRICULAPROPIO                                0
                                                       ..
FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta    0
RENDIMIENTO_GLOBAL_alto                                 0
RENDIMIENTO_GLOBAL_bajo                                 0
RENDIMIENTO_GLOBAL_medio-alto                           0
RENDIMIENTO_GLOBAL_medio-bajo                           0
Length: 1038, dtype: int64

Valor máximo de las columnas numéricas después de la escalación:
 FAMI_TIENEINTERNET                                      1
FAMI_TIENELAVADORA                                      1
FAMI_TIENEAUTOMOVIL                                     1
ESTU_PRIVADO_LIBERTAD       

In [28]:
data_frame

,ID,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,coef_1,...,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_alto,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,904256,20212,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1,645256,20212,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
2,308367,20203,1,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
3,470353,20195,1,1,0,0,0,1,1,0,...,0,0,1,0,0,0,1,0,0,0
4,989032,20212,1,1,1,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,1,1,0,0,1,1,1,0,...,0,0,0,1,0,0,0,0,1,0
692496,754213,20212,1,1,0,0,0,1,1,0,...,0,0,0,1,0,0,0,1,0,0
692497,504185,20183,1,1,0,0,1,1,1,0,...,0,0,0,1,0,0,0,0,0,1
692498,986620,20195,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,1,0,0
